### Jul AdventKalender D20

https://adventofcode.com/2024/day/20

#### Part 1

In [86]:
import numpy as np

def read_file():
    input_file_path = "data/input20.txt"
    race_map = []
    def process_line(line):
        race_map.append(list(line.strip()))

    with open(input_file_path, 'r') as file:
        for line in file:
            process_line(line)
            
    return np.array(race_map)

race_map = read_file()
pos_start, pos_end = tuple(i[0] for i in np.where(race_map=="S")), tuple(i[0] for i in np.where(race_map=="E"))

In [87]:
import heapq
move = [(0, 1), (1, 0), (0, -1), (-1, 0)] # [move right, down, left, up]
rows, cols = race_map.shape

def is_in_map(x, y):
    return 0 <= x < rows and 0 <= y < cols

def is_valid_move(x, y):
    return is_in_map(x, y) and race_map[x, y]!="#"

def is_found_goal(x, y):
    return x == pos_end[0] and y == pos_end[1]

def Dijkstra_find_path():
    visited = set() # pos=(x, y), move_direction
    predecessors = {}  # to track paths: (pos, direction) -> [(previous_pos, previous_direction)]

    todolist = []
    for direction_init in [0,1,2,3]:
        if is_valid_move(pos_start[0] + move[direction_init][0], pos_start[1] + move[direction_init][1]):
            todolist.append((0, pos_start, direction_init)) # todolist: cost, pos=(x, y), move_direction
    
    while(todolist):
        cost, pos, direction = heapq.heappop(todolist) # pop the smallest item(smallest cost)
        if (pos, direction) in visited:
            continue
        visited.add((pos, direction))
        
        if is_found_goal(pos[0], pos[1]):
            return cost, predecessors
            
        nx, ny = pos[0] + move[direction][0], pos[1] + move[direction][1]
        if is_valid_move(nx, ny):
#             print(nx, ny)
            new_cost = cost + 1 # for moving forward
            for direction_prepare in [0,1,2,3]: # for todolist
                if move[direction_prepare][0] == -move[direction][0] and move[direction_prepare][1] == -move[direction][1]:
                    continue  # not go back
                heapq.heappush(todolist, (new_cost, (nx, ny), direction_prepare))
                
                # add predecessors
                if ((nx, ny), direction_prepare, new_cost) not in predecessors:
                    predecessors[((nx, ny), direction_prepare, new_cost)] = []
                predecessors[((nx, ny), direction_prepare, new_cost)].append((pos, direction, cost))
    return None, None

In [88]:
def reconstruct_paths(optimal_cost):
    paths = []
#     positions = set()
    todolist = []
    for direction in [0,1,2,3]:
        px, py = pos_end[0] - move[direction][0], pos_end[1] - move[direction][1]
        if is_valid_move(px, py):
            todolist.append([(pos_end, direction, optimal_cost)]) # todolist: paths, each path is a state: pos=(x, y), dir, cost

    def if_start(state):
        if state[0] == pos_start and state[2] == 0:
                return True
        return False

#     print(todolist)
    while todolist:
#         print(todolist)
        path = todolist.pop()
        current = path[-1]
        
        if if_start(current):
#             for state in path:
#                 positions.add(state[0])
            paths.append(path[::-1])  # Reverse path for correct order
#             print(paths)
            continue

        for parent in predecessors[current]:
            if parent not in path and parent[-1] < current[-1]:
                todolist.append(path + [parent])
    return paths

In [89]:
cost, predecessors = Dijkstra_find_path()
path_nocheat = reconstruct_paths(cost)
path_nocheat = [state[0] for state in path_nocheat[0]]

In [90]:
total_cheats = 0
for i in range(len(path_nocheat)):
    for j in range(i, len(path_nocheat)):
        node_left, node_right = path_nocheat[i], path_nocheat[j]
        if abs(node_left[0]-node_right[0])+abs(node_left[1]-node_right[1]) == 2 and (j-i >= 2 + 100):
            total_cheats += 1
total_cheats

1490

#### Part 2

In [92]:
total_cheats = 0
for i in range(len(path_nocheat)):
    for j in range(i, len(path_nocheat)):
        node_left, node_right = path_nocheat[i], path_nocheat[j]
        distance = abs(node_left[0]-node_right[0])+abs(node_left[1]-node_right[1])
        if 2 <= distance <= 20 and (j-i >= distance + 100):
            total_cheats += 1
total_cheats

1011325